In [2]:
import pandas as pd
!pip install pdfquery
import pdfquery
import os
import numpy as np
import math
import requests
from os import sep
from google.colab import drive
import glob
from bs4 import BeautifulSoup
#https://www.freecodecamp.org/news/extract-data-from-pdf-files-with-python/

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 95.1 MB/s eta 0:00:00
  Created wheel for pdfquery: filename=pdfquery-0.4.3-py3-none-any.whl size=16781 sha256=34386e348e92e763039ff3fccfed206591bae2bff2d32839535c33b89ddb0605
  Stored in directory: /root/.cache/pip/wheels/98/a2/41/ca6652543d0fa5762560eaaf0f620a5d6341ec0b9e60996d16
Successfully built pdfquery


In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd /content/drive/MyDrive/Rafel-Satellitecollision-GSoC2023/data/collision/
%ls

/content/drive/.shortcut-targets-by-id/1NyJDOWUmDY1vQY1mod9dhwk8XC_AaT3-/Rafel-Satellitecollision-GSoC2023/data/collision
collisions.xml    sat_col_data_table.gsheet
sat_col_data.csv  Satellite_collisions.pdf


In [5]:
pdf = pdfquery.PDFQuery('Satellite_collisions.pdf')

In [34]:
collision_df = pd.read_csv('sat_col_data.csv')

In [31]:
collision_df.isna()

In [32]:
collision_df['additional_information'] = collision_df['additional_information'].replace('SELFDESTRUCT', 'SELF DESTRUCT')

In [35]:
collision_df.head()

,satellite_name,international_designator,us_satellite_number,launch_date,breakup_date,debris_cataloged,debris_left,apogee,perigee,inclination,assessed_cause,additional_information,location,date,time
0,TRANSIT 4A R/B,1961-015C,118,29-Jun-61,29-Jun-61,317,193,995.0,880.0,66.8,PROPULSION,ABLESTAR STAGE,28N 254E (dsc),29 Jun 1961,0608 GMT
1,SPUTNIK 29,1962-057A,443,24-Oct-62,29-Oct-62,24,0,260.0,200.0,65.1,PROPULSION,MOLNIYA FINAL STAGE,NaN,NaN,NaN
2,ATLAS CENTAUR 2,1963-047A,694,27-Nov-63,27-Nov-63,19,8,1785.0,475.0,30.3,PROPULSION,CENTAUR STAGE,NaN,NaN,NaN
3,COSMOS 50,1964-070A,919,28-Oct-64,5-Nov-64,96,0,220.0,175.0,51.2,DELIBERATE,SELFDESTRUCT,NaN,NaN,NaN
4,COSMOS 57,1965-012A,1093,22-Feb-65,22-Feb-65,167,0,425.0,165.0,64.8,DELIBERATE,SELFDESTRUCT,NaN,NaN,NaN


In [36]:
collision_df['satellite_name'].value_counts()

SL-23 DEBRIS                      2
COSMOS 2392 ULLAGE MOTOR          2
COSMOS 2079- 81 ULLAGE MOTOR      2
COSMOS 1970- 72 ULLAGE MOTOR      2
COSMOS 1987- 89 ULLAGE MOTOR      2
                                 ..
COSMOS 1375                       1
COSMOS 1691 (1695)                1
COSMOS 1714 R/B                   1
NOAA 8                            1
COSMOS 2434- 2436 ULLAGE MOTOR    1
Name: satellite_name, Length: 255, dtype: int64

In [38]:
dates =  ['29 Jun 1961', '29 Oct 1962', '27 Nov 1963', '5 Nov 1964', '22 Feb 1965', '15 Mar 1965', '15 Jan 1966', '15 Oct 1965', '21 Dec 1965', '15 Feb 1966', 'Mid-Jun 1966', '12 Jul 1975', '20 Jan 1976', '5 Jul 1966', '17 Sep 1966', '2 Nov 1966', 'TIME:', '24 Jan 1968', '13 Apr 1968', '21 Feb 1992', '1 Nov 1968', '20 Oct 1968', '1 Nov 1968', '15 Nov 1973', 'TIME:', '28 Mar 1969', '26 Jul 1969', '4 Oct 1969', '17 Oct 1970', '23 Oct 1970', '30 Oct 1970', '25 Feb 1971', '3 Dec 1971', '22 May 1975', '3 Apr 1973', '6 May 1973', '28 Dec 1973', '17 Jan 2005', '20 Aug 1975', '17 Apr 1975', '2 Aug 1975', '9 Feb 1976', '19 Jun 1976', '1 May 1991', '6 Sep 1975', '25 Jan 1976', 'TIME:', '17 May 1977', '29 Sep 1977', '25 Jul 1976', '24 Dec 1977', '15 Mar 1977', '27 Nov 1978', '29 Dec 1976', '27 Dec 1976', '8 Jun 1978', '30 March 197', '14 Jul 1977', '24 Oct 1977', '23 Jun 1978', '21 Dec 1977', '27 Jan 1981', '10 Oct 1978', '26 Dec 1981', '9 May 1988', '13 Sep 1985', '17 Sep 1979', 'Mid-Feb 1980', '9 Sep 1979', '23 Apr 1981', '~ 11 Dec 197', 'Apr 1980', '15 Jul 1981', '18 Apr 1980', '26 August 19', '14 May 1981', '12 Feb 1983', '20 Jun 1982', '25 Aug 1982', '20 Oct 1981', 'LOCATION:', 'LOCATION:', 'Apr-May 1981', '24 Jul 1981', 'Early Dec 19', '21 Nov 1981', '29 Sep 1982', '11 Sep 1981', '12 Jul 1982', '18 Sep 1982', '25-28 Jan 19', '2 September ', 'LOCATION:', 'LOCATION:', 'LOCATION:', '21 Oct 1985', '20 Dec 1983', '15 Nov. 2021', '23 Dec 1983', '8 Dec 1982', '3 Sep 1984', '30 Dec 1985', '13 Aug 1983', '11 Mar 1985', '13 May 1985', '9 Jul 1983', '18 Oct 1993', '4 Feb 1991', '6 Feb 1984', '3 Feb 1984', '23 Feb 1986', '5 Sep 1992', '20 Nov 1984', '20 Nov 1987', '29 Nov 1998', '21 Jun 1985', '5 Jan 1988', '18 Dec 1986', '22 Nov 1985', '4 May 2006', '28 Dec 1985', '29 Dec 1991', '13 Nov 1986', '21 Sep 1987', '5 Sep 1986', '5 Sep 1986', '29 Jan 1987', '17 Dec 1987', '26 Jul 1987', '27 Nov 1997', '15 Feb 1998', '16-19 Sep 19', '\uf07e 01 Decembe', '23 Apr 2003', '31 Jan 1988', '22 May 1997', '21 June 2005', '14-17 Decemb', '27 Feb 1988', '23 Dec 1991', 'TIME:', '04 Aug 2003', '9 Mar 1999', '17 Feb 1998', '3 Aug 1998', '13 Nov 2003', '17-18 Dec 19', '~1 Jan 2001', 'TIME:', '12 Jan 1993', '28 Jul 1989', '31 Aug 1989', '3 Dec 2006', '18 Apr 1999', 'Jul 1992 (?)', 'TIME:', '~28 Mar 1999', '4 Oct 1990', '30 Nov 1990', 'Approx. 14 D', '1 Dec 1990', '21 Feb 2003', '14 Mar 1998', 'Late Apr-ear', '5 Mar 1991', '7 May 1994', '27 Apr 1994', 'TIME:', '16 Jun 2001', 'TIME:', '09 Oct 1999', '6 December 1', '24 Dec 2001', '15 Apr 2004', '21 Apr 1993 ', 'Feb 2002', 'TIME:', 'Prior to 082', 'TIME:', '14 Jul 2001', '~22 Apr 2005', '18 Feb 1993', '26 Dec 1992', '26 Dec 1992', '26 Dec 1992', '30 Dec 1992', '28 Mar 1993', '1 Dec 1994', '27 Apr 1993', 'TIME:', '25 Jul 1993', '18 Dec 1993', '6-7 Sep 2000', '7 Feb 1994', '3 Jun 1996', 'Approx. 21 O', '', '27 Aug. 2020', '7 June 2014', '26 Dec 1994', '3.736 Feb 20', '26 June 1997', '24 Jul 1996', '21 Nov 2000', '19 Feb 1996', '~13 Dec 1999', '16 Sep 1997', 'TIME:', '14 Feb 2007', '22 Nov 1999', '18 Nov. 2021', '25 Dec 1997', '21 Feb 1998', '22 Dec. 2018', '28 April 201', '11 Jan 2007', '18 Feb 2007', '11 Mar 2000', '21 Nov 2001', '~1 Sep 2006', '25 Nov 2015', '19 Dec 2001', '28 Feb 2004', '10 Mar. 2021', '1 Jun 2005', '29 Oct 2004', 'TIME:', 'TIME:', '19 Feb 2007', 'TIME:', 'TIME:', '28 Dec 2006', '17 Nov 2006', '4 Nov 2006', 'TIME:', 'TIME:', 'TIME:', '02 Feb 2007', '23 Dec 2010', '10 May 2014', 'TIME:', '11 Nov 2007', '15 Apr. 2022', '18 Aug 2011', 'TIME:', 'TIME:', 'TIME:', 'TIME:', '29 June 2016', '21-23 Dec. 2', '9 July 2014', 'TIME:', 'TIME:', 'TIME:', '~4 Sep 2010', 'TIME:', '21 Dec 2011', '26 Feb 2012', 'TIME:', 'TIME:', 'TIME:', 'TIME:', 'TIME:', '23 Aug 2016', 'TIME:', '31 Aug. 2018', '28 April 201', 'TIME:', 'TIME:', 'TIME:', 'TIME:', 'TIME:', '27 Mar. 2019', '9 Jan. 2020', '18 Mar. 2021', '26 Nov. 2021']
locations = ['28N, 254E (dsc)', 'Unknown', 'Unknown', 'Unknown', '64N, 80E (asc)', '51S, 162E (dsc)', 'Unknown', '22S, 108E (asc)', 'unknown', 'Unknown', 'Unknown', '67N, 135E (dsc)', 'Unknown', '20N, 277E (dsc)', 'Unknown', 'Unknown', '2150 GMT', 'Unknown', '32N, 245E (asc)', 'Unknown (~ 197E)', '55N, 104E (dsc)', '57S, 181E (asc)', '58N, 34E (asc)', 'Unknown', '2101 GMT', '59N, 91E (dsc)', '0N, 333E (dsc)', '54N, 178E (dsc)', '50S, 142E (asc)', '22S, 217E (asc)', '54N, 23E (asc)', '54N, 21E (asc)', '51N, 7E (asc)', '34S, 46E (asc)', '45N, 290E (dsc)', '71S, 215E (asc)', '37S, 181E (asc)', '80.6S, 306.8E', '52S, 278E (dsc)', '01N, 278E (dsc)', '02S, 258E (dsc)', 'Unknown', '7N, 344E (dsc)', '66N, 322E (asc)', '32N, 293E (asc)', '53N, 7E (asc)', '1902 GMT', '9S, 284E (dsc)', '33S, 162E (dsc)', '49N, 100E (dsc)', '40S, 146E (asc)', '39N, 114E (asc)', '65S, 306E (dsc)', 'Unknown', '65S, 210E (asc)', 'Unknown', '63S, 0E (dsc)', '14N, 249E (dsc)', 'Unknown', '0.0N, 98.7E', '38S, 274E (asc)', '80S, 301E (asc)', 'Unknown', 'Unknown', '29S, 126E (dsc)', '35N, 234E (asc)', '53S, 336E (dsc)', 'Unknown', '52N, 304E (asc)', 'Unknown', 'Unknown', 'Unknown', '10N, 106E (asc)', '47N, 322E (asc)', 'Unknown', 'Unknown', 'Unknown', '10S, 332E (dsc)', '65S, 238E (dsc)', 'Unknown', 'TIME:', 'TIME:', 'Unknown', '68N, 197E (asc)', 'Unknown', 'Unknown', '51N, 80E (asc)', 'Unknown', '65S, 40E (asc)', '32N, 293E (asc)', 'Unknown', '39.5N, 121.3E (asc)', 'TIME:', 'TIME:', 'TIME:', '66N, 351E (asc)', '25S, 45E (dsc)', '70.25 N, 48.39 E (as', 'Unknown', '62S, 302E (asc)', '12S, 352E (dsc)', '68S, 300E (dsc)', 'Unknown', '4S, 196E (asc)', '10N, 82E (asc)', 'Unknown', '7S, 111E (asc)', '28N, 106E (dsc)', '0N, 120E (asc)', '0N, 56E (asc)', '29N, 187E (asc)', '46.1S, 351.8E', 'Unknown', '65N, 300E (dsc)', '38.3N, 172.6E', '8N, 292E (asc)', '66N, 151E (asc)', '22S, 292 E (asc)', '31N, 326E (dsc)', '67N, 17E (dsc)', 'Unknown', '25.3N, 331.9E', '7N, 42E (asc)', '60S, 174E (dsc)', '15N, 166E (asc)', '15N, 166E (asc)', '73N, 122E (asc)', '15S, 18E (dsc)', '57S, 239E (asc)', 'Unknown', '67.8 N, 125.6 E (asc', 'Unknown', 'Unknown', 'Unknown', '11S, 138E (dsc)', 'Unknown', '36.5N, 74.22E (asc)', 'Unknown', '62N, 98E (asc)', 'Unknown', '1930Z', 'Unknown', 'Unknown', '6.9 N, 157.2 E (dsc)', 'Unknown', 'Unknown', 'Unknown', 'Unknown', '1320 GMT', 'Unknown', '35-65N, 95-140E (asc', '43N, 111E (dsc)', '52S, 168E (dsc)', '16.9S, 234.1E', 'Unknown', '0344 GMT', 'Unknown', '81S, 68E (asc)', '54N, 157E (dsc)', 'Unknown', '6N, 232E (dsc)', '34.11 S, 208.61 E', 'Unknown', 'Unknown', '43S, 140E (asc)', '10N, 112E', '0.5S, 79E (dsc)', '1000 GMT', 'Unknown', '1046 GMT', '41.5N, 217.9E (dsc)', '55N, 154E (dsc)', '4.0 N, 344.4 E (asc)', '31N, 56E (asc)', 'Unknown', 'Unknown', '0240 GMT (est.)', 'Unknown', '1902 Z', 'Unknown', 'Unknown', '55N, 157E (dsc)', '63 N, 60 E (asc)', '44 N, 168 E (asc)', '52 N, 63 E (dsc)', '22 S, 172 E (dsc)', '70N, 37E (dsc)', '6.5 S, 243.0 E (dsc)', '61N, 81E (asc)', '1656 GMT', 'Unknown', '65N, 107E (dsc)', 'Unknown', '59S, 234E (dsc)', '67 S, 56 E (asc)', 'Unknown', '', 'Unknown', 'unknown', '51.6S, 307E (asc)', '40.8S, 7.5E', '44 N, 173 E (asc.)', '38 S, 60 E (asc)', 'Unknown', '0.2 N, 88.8 E (dsc)', 'Unknown', '58.2 N, 157.5 E (asc', '1656 GMT', 'Unknown', '31.6N, 4.3E (dsc)', 'Unknown', '0.3 S, 91.2 E (dsc)', 'Unknown', 'Unknown', 'unknown', '35N, 100E (asc)', '35N, 128E (asc)', '51.2S, 311.5W', '38.3 S, 342.3 E (dsc', 'Unknown', '77.5 N, 48.8 E (dsc)', '25 S, 340 E (asc)', '26S, 100E (dsc)', '5.99 N, 281.47 E (as', 'Unknown', 'Unknown', '0129Z', '2159 GMT', '31S, 135E (asc)', '0649 GMT', '0429 GMT', '58S, 131E (asc)', 'Unknown', 'Unknown', '0326 GMT', '0119 GMT', '1200 GMT', 'Unknown', 'Unknown', 'TBD', '1340 GMT', 'Unknown', 'Unknown', 'Unknown', '0553 GMT', '0032 GMT', '1212 GMT', '0925 GMT', 'unknown', 'Unknown', '33.96N, 41.3E (asc)', '0206 GMT', '0237 GMT', '1358 GMT', 'Unknown', '1731 GMT', 'Unknown', 'Unknown', '1600 GMT', '1631 GMT', '1641 GMT', '0846 GMT', '1302 GMT', '72.03S, 39.59E (dsc)', '1814 GMT', 'Unknown', 'TBD', '0350 GMT', '0142 GMT', '0957 GMT', '1859 GMT', '1000 GMT', '10.25 N, 89.02 E', '80.52 N, 221.58 E', '71.99 N, 21.19 E (ds', '39.81 N, 17.80 E (ds']
times = ['0608 GMT', 'Unknown', 'Unknown', 'Unknown', '', '1714 GMT', 'Unknown', '1820 GMT', 'unknown', 'Unknown', 'Unknown', '2248 GMT', 'Unknown', '2111 GMT', 'Unknown', 'Unknown', 'ASSESSED CAU', 'Unknown', '1054 GMT', '0931 GMT', '0412 GMT', '1427 GMT', '0402 GMT', 'Unknown', 'ASSESSED CAU', '1845 GMT', '0228 GMT', '1553 GMT', '0317 GMT', '1513 GMT', '0600 GMT', '1431 GMT', '1651 GMT', '1827 GMT', '2236 GMT', '0724 GMT', '0904 GMT', '0214 GMT', '1307 GMT', '2148 GMT', '1623 GMT', 'Unknown', '0659 GMT', '0856 GMT', '1906 GMT', '1400 GMT', 'ASSESSED CAU', '1018 GMT', '0717 GMT', '1718 GMT', '1133 GMT', '1256 GMT', '1703 GMT', 'Unknown', '1840 GMT', 'Unknown', '1545 GMT', '1612 GMT', 'Unknown', 'Unknown', '1710 GMT', '0432 GMT', 'Unknown', 'Unknown', '1218 GMT', '2043 GMT', '1039 GMT', 'Unknown', '0230 GMT', 'Unknown', 'Unknown', 'Unknown', '0921 GMT', '0726 GMT', 'Unknown', 'Unknown', 'Unknown', '1818 GMT', '1231 GMT', 'Unknown', 'ASSESSED CAU', 'ASSESSED CAU', 'Unknown', '2351 GMT', 'Unknown', 'Unknown', '0520 GMT', 'Unknown', '2325 GMT', '1702 GMT', 'Unknown', '0957 GMT', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', '0353 GMT', '1215 GMT', '0248 GMT', 'Unknown', '1448 GMT', '2023 GMT', '1005 GMT', 'Unknown', '0940 GMT', '0133 GMT', 'Unknown', '1204 GMT', '0312 GMT', '1600 GMT', '2145 GMT', '1850 GMT', '1451 GMT', '1425Z', '0131 GMT', '0925 GMT', '1047 GMT', '0147 GMT', '2017 GMT', '0840 GMT', '1604 GMT', 'Unknown', '0903 GMT', '1940 GMT', '1205 GMT', '1752 GMT', '1752 GMT', '0555 GMT', '1739 GMT', '1539 GMT', 'ALTITUDE:', '2224 GMT', 'Unknown', 'Unknown', '~1800Z', '1109 GMT', 'Unknown', '09:14:54.5 G', 'Unknown', '0444 GMT', 'Unknown', 'ASSESSED CAU', '~0725Z', 'Unknown', '1235 GMT', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'ASSESSED CAU', 'Unknown', '0410-0420 GM', '1851 GMT', '0337 GMT', '0119 GMT', 'Unknown', 'ASSESSED CAU', 'Unknown', '2014 GMT', '1720 GMT', 'Unknown', '1610 GMT', '~0300Z', 'Unknown', 'Unknown', '1345 GMT', '0930 GMT', '0144 GMT', 'ASSESSED CAU', '~0700Z', 'ASSESSED CAU', '1508 GMT', '2021 GMT', '~2212Z', '1454 GMT', 'Unknown', 'Unknown', 'ASSESSED CAU', 'Unknown', 'ASSESSED CAU', '~1800Z', 'Unknown', '1856 GMT', '0738 GMT', '2249 GMT', '2310 GMT', '0903 GMT', '0716 GMT', '1111 GMT', '1044 GMT', 'ASSESSED CAU', 'Unknown', '0711 GMT', 'between 1918', '1719 GMT', '1518 GMT', 'Unknown', '95130.000879', '1000 GMT ± 8', 'unknown', '0627 GMT', '1740 GMT', '0257 GMT', '0948 GMT', 'Unknown', '1459 GMT', 'Unknown', '2208 GMT', 'ASSESSED CAU', 'Unknown', '0440 GMT', '0600-0700 GM', '0550 GMT', 'Unknown', '~ 0712 GMT', 'unknown', '2226 GMT', '1107 GMT', '1304 GMT', '1414Z', 'Unknown', '0950 GMT', '~1140Z', '~1930 GMT', '0711 GMT', 'Unknown', 'Unknown', 'ASSESSED CAU', 'ASSESSED CAU', '1721 GMT', 'ASSESSED CAU', 'ASSESSED CAU', '1729 GMT', '~1800 GMT', 'Unknown', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'Unknown', '1600 GMT', '2024 GMT', 'ASSESSED CAU', 'Unknown', '0520-2033 GM', 'Unknown', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'unknown', 'Unknown', '0828 GMT', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'Unknown', 'ASSESSED CAU', 'Unknown', 'Unknown', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', '1707 GMT', 'ASSESSED CAU', '0200 GMT', '07:18:35 GMT', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', 'ASSESSED CAU', '0540 GMT', '2200 GMT', '0741 GMT', '2245 GMT']

collision_df['date'] = dates[:268]
collision_df['location'] = locations[:268]
collision_df['time'] = times[:268]



In [39]:
collision_df.tail(10)

,satellite_name,international_designator,us_satellite_number,launch_date,breakup_date,debris_cataloged,debris_left,apogee,perigee,inclination,assessed_cause,additional_information,location,date,time
258,SL-23 DEBRIS,2011-037B,37756,18-Jul-11,8-May-20,334,282,3606.0,422.0,51.5,UNKNOWN,Fregat-SB SBB propellant tank,1600 GMT,TIME:,ASSESSED CAU
259,H-2A DEB,2018-084C,43673,29-Oct-18,12-Jul-20,88,3,643.0,595.0,97.9,UNKNOWN,Payload Encapsulation System (PES) lateral cyl...,1631 GMT,TIME:,ASSESSED CAU
260,RESURS O1,1994-074A,23342,4-Nov-94,27-Aug-20,141,140,660.0,633.0,97.9,UNKNOWN,Resurs-O No. 3,1641 GMT,TIME:,ASSESSED CAU
261,NOAA 17,2002-032A,27453,24-Jun-02,10-Mar-21,115,115,817.0,800.0,98.62,UNKNOWN,NaN,0846 GMT,TIME:,ASSESSED CAU
262,YUNHAI 1-02,2019-063A,44547,25-Sep-19,18-Mar-21,38,32,785.0,780.0,98.54,COLLISION,ACCIDENTAL HIT BY DEBRIS (48078),1302 GMT,TIME:,ASSESSED CAU
263,COSMOS 2499,2014-028E,39765,23-May-14,23-Oct-21,23,23,1507.0,1152.0,82.44,UNKNOWN,"2nd ""SATELLITE INSPECTOR""","72.03S, 39.59E (dsc)",23 Aug 2016,1707 GMT
264,COSMOS 1408,1982-092A,13552,16-Sep-82,15-Nov-21,1760,990,490.0,465.0,82.6,COLLISION,DELIBERATE Russian ASAT test,1814 GMT,TIME:,ASSESSED CAU
265,ORBCOMM FM 5,1997-084F,25117,23-Dec-97,18-Nov-21,15,11,771.0,758.0,45,UNKNOWN,NaN,Unknown,31 Aug. 2018,0200 GMT
266,USA 305-308 MINOTAUR 4 R/B,2020-046E,45877,15-Jul-20,26-Nov-21,22,17,580.0,567.0,54,UNKNOWN,NaN,TBD,28 April 201,07:18:35 GMT
267,COSMOS 2434- 2436 ULLAGE MOTOR,2007-065F,32398,25-Dec-97,15-Apr-22,1,1,19068.0,400.0,64.8,PROPULSION,PROTON-K BLOCK DM SOZ,0350 GMT,TIME:,ASSESSED CAU


In [40]:
collision_df['breakup_date'].value_counts()

26-Mar-16    2
10-Feb-09    2
5-Sep-86     2
1-Nov-68     2
17-Nov-06    1
            ..
23-Feb-86    1
13-Nov-86    1
Dec-86       1
18-Dec-86    1
15-Apr-22    1
Name: breakup_date, Length: 264, dtype: int64

In [41]:
rows_with_nan = collision_df[collision_df['additional_information'].isnull()]
print(rows_with_nan)

                 satellite_name international_designator  us_satellite_number  \
8                     COSMOS 95                1965-088A                 1706   
44                   COSMOS 839                1976-067A                 9011   
49                      EKRAN 2                1977-092A                10365   
51                   COSMOS 880                1976-120A                 9601   
62                      EKRAN 4                1979-087A                11561   
65                  COSMOS 1275                1981-053A                12504   
80                      EKRAN 9                1982-093A                13554   
90                  COSMOS 1375                1982-055A                13259   
91           COSMOS 1691 (1695)                1985-094B                16139   
93                       NOAA 8                1983-022A                13923   
105                 COSMOS 1823                1987-020A                17535   
109             COSMOS 1045 

In [42]:
collision_df.to_csv('collision_satellites.csv')

In [43]:
%ls

collision_satellites.csv  sat_col_data.csv           Satellite_collisions.pdf
collisions.xml            sat_col_data_table.gsheet
